In [1]:
from ipywidgets import widgets
import numpy as np
from time import time, sleep
from os import system
from h5py import File as h5py_File
import matplotlib.pyplot as plt
from notebook.notebookapp import list_running_servers

In [4]:
width_png=547
height_png=400

In [5]:
image_height=250
image_width=int(width_png*image_height/height_png)
raw_bytes_width=100
tests_results_width=300
tmp_image_name='tmp_ex2.svg'

In [ ]:
s_data_file_name='data_for_ex2'

In [ ]:
fid_host_url=open('host_url.txt','r')
v_lines_host_url=fid_host_url.readlines()
fid_host_url.close()
s_line_host_url=v_lines_host_url[0]
if s_line_host_url[-1]=='\n':
    s_host_url=s_line_host_url[:-1]
else:
    s_host_url=s_line_host_url[:]

In [ ]:
a=system('unzip data/%s.zip -d . >> test.log'%s_data_file_name)

In [ ]:
servers = list(list_running_servers())
s_base_url=servers[0]['base_url']

In [3]:
hdf5_file = '%s.h5'%s_data_file_name
data_file = h5py_File(hdf5_file,'r')
v_ratio_period_sample_for_interact=np.array(data_file['parameters/v_ratio_period_sample'][0,:],dtype=np.uint32)
v_jitter_for_interact=data_file['parameters/v_jitter'][0,:]
v_raw_rng_image=data_file['images/rng_image'][0,:]
m_raw_bits_from_one_RO=data_file['raw_bits/m_raw_bits_from_one_RO'][:]
data_file.close()
system('rm -Rf %s'%hdf5_file)
nb_test_per_corner=m_raw_bits_from_one_RO.shape[3]
nb_bits=m_raw_bits_from_one_RO.shape[4]
nb_osc=m_raw_bits_from_one_RO.shape[0]

In [6]:
v_x=np.array([ 0,  1,  2,  4,  8,  3,  5,  6,  9, 10, 12,  7, 11, 13, 14, 15], dtype=np.uint8)
v_s_x=['0000', '0001', '0010', '0100', '1000', '0011', '0101', '0110', '1001', '1010', '1100', '0111', '1011', '1101', '1110', '1111']

def generate_test_image(v_nibble):
    
    v_hist=np.zeros((16,),dtype=np.uint32)
    for i in range(16):
        v_hist[i]=((v_nibble==v_x[i]).sum())
        
    nb_tau=100
    nb_words_to_be_used=v_nibble.shape[0]-nb_tau
    v_T5=np.zeros((nb_tau,),dtype=np.float64)
    for tau in range(nb_tau):
        v_T5[tau]=(((v_nibble[:nb_words_to_be_used])^(v_nibble[tau+1:tau+nb_words_to_be_used+1])).mean())-7.5
    
    fig, ax = plt.subplots(2, 1)
    ax[0].bar(np.arange(16),v_hist)
    ax[0].set_title('4-bits words distribution')
    ax[0].set_xlabel('4-bits word value')
    ax[0].set_xticks(np.arange(16))
    ax[0].set_xticklabels(v_s_x,rotation='vertical')
    
    ax[0].plot(np.arange(17)-0.5,np.ones(17)*370,'r')
    ax[0].plot(np.arange(17)-0.5,np.ones(17)*254,'r')
    
    
    ax[1].plot(v_T5)
    ax[1].set_xlabel('Lag')
    ax[1].set_title('4-bits words Autocorrelation')
    ax[1].plot(np.arange(nb_tau),np.ones(nb_tau)*0.23,'r')
    ax[1].plot(np.arange(nb_tau),np.ones(nb_tau)*(-0.23),'r')
        
    fig.tight_layout(pad=0, h_pad=2.0, w_pad=None)
                   
    plt.savefig(tmp_image_name,pad_inches=0)
    plt.close(fig)
    fid_image_hist=open(tmp_image_name,'rb')
    v_raw_image_hist=fid_image_hist.read()
    fid_image_hist.close()
    return v_raw_image_hist

In [7]:
v_raw_image_hist=generate_test_image(np.zeros((nb_bits,),dtype=np.uint8))

sample_period_value_widgets=widgets.SelectionSlider(description=r'\(D\)',options=list(v_ratio_period_sample_for_interact),layout=widgets.Layout(width='%dpx'%(image_height+raw_bytes_width)),readout_format='.1f')
m_value_widgets=widgets.SelectionSlider(description=r'\(m\)',options=list(np.arange(nb_osc,dtype=np.uint8)+1),layout=widgets.Layout(width='%dpx'%(image_height+raw_bytes_width)))
std_dev_value_widgets=widgets.SelectionSlider(description=r'\(\frac{\sigma_{tot}}{T_{1}}\)',options=list(v_jitter_for_interact),layout=widgets.Layout(width='%dpx'%(image_height+raw_bytes_width)))

rng_image=widgets.Image(value=v_raw_rng_image.tobytes(), format='png',layout=widgets.Layout(height='%dpx'%(image_height),width='%dpx'%image_width))
hist_image=widgets.Image(value=v_raw_image_hist, format='svg+xml',layout=widgets.Layout(height='%dpx'%(image_height),width='auto'))
raw_bytes=widgets.Textarea(value='',rows=5000,placeholder='',description='',disabled=True,layout=widgets.Layout(height='%dpx'%(image_height),width='%dpx'%(image_width/4)))

float_bar=widgets.FloatProgress(value=0.0,min=0.0, max=100.0,description='',bar_style='info',style={'bar_color': '#ff0000'},orientation='horizontal',layout=widgets.Layout(width='%dpx'%tests_results_width))
val_button=widgets.Button(description = 'GENERATE RANDOM VALUES',layout=widgets.Layout(width='%dpx'%tests_results_width))
val_button.style.button_color = 'gray'
html_download=widgets.HTML(value='',layout=widgets.Layout(width='%dpx'%tests_results_width))

def generate_rnd(b):
    raw_bytes.value=''
    chosen_sample_period=sample_period_value_widgets.value
    chosen_jitter=std_dev_value_widgets.value
    chosen_m=m_value_widgets.value    
    
    index_sample_period=np.where(chosen_sample_period==v_ratio_period_sample_for_interact)[0][0]
    index_chosen_jitter=np.where(chosen_jitter==v_jitter_for_interact)[0][0]

    v_raw_bits=np.zeros((nb_bits,),dtype=np.bool)
    for i in range(chosen_m):
        tmp_rand=np.random.randint(0,nb_test_per_corner,dtype=np.uint32)
        v_raw_bits^=m_raw_bits_from_one_RO[i-1,index_chosen_jitter,index_sample_period,tmp_rand,:]
        
    nb_nibble=int((nb_bits)/4)
    v_nibble=np.zeros((nb_nibble,),dtype=np.uint8)
    for i in range(4):
        v_nibble+=np.array(v_raw_bits[i:4*nb_nibble:4],dtype=np.uint8)*(2**i)
    html_download.value=''
    s_line=''
    for i in range(nb_nibble):
        s_line='%s%X'%(s_line,v_nibble[i])
    raw_bytes.value=s_line   
    
    randomvalues_file_name='randomvalues.txt'
    fid_rnd_file=open(randomvalues_file_name,'w')
    fid_rnd_file.write(s_line)
    fid_rnd_file.close()
    path_to_download='%s%sfiles/%s'%(s_host_url,s_base_url,randomvalues_file_name)
    s_line_for_download='<p><a href="%s" target="_blank" rel="noopener noreferrer">  Click here to download the random values</a></p>'%(path_to_download)
    hist_image.value=generate_test_image(v_nibble)
    
    float_bar.value=0
    time_to_sleep=0.01
    for i in range(100):
        sleep(time_to_sleep)
        float_bar.value=(i+1)
    html_download.value=s_line_for_download
val_button.on_click(generate_rnd, remove=False) 
v_box_top=widgets.VBox([widgets.HBox([rng_image,raw_bytes,hist_image]),widgets.VBox([widgets.HBox([widgets.VBox([m_value_widgets,std_dev_value_widgets,sample_period_value_widgets]),widgets.VBox([val_button,float_bar,html_download])])])])
display(v_box_top)

Widget Javascript not detected.  It may not be installed or enabled properly.
